In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from sklearn.metrics import accuracy_score, hamming_loss, jaccard_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform, loguniform
from scipy.stats import randint
import pandas as pd
import scipy as sp
import seaborn as sns

# Problem transformation

## Difference Between Binary Relevance Approach and Classifier Chains Approach

In the binary relevance approach, a multi-classification problem is broken down into multiple binary classification sup-problems by creating a classifier for each label. On the other hand, in the classifier chains approach, a sequence of binary classifiers are created but, unlike the binary relevance approach, the classifiers are not independent as predictions from the previous classifiers are used as inputs for the next classifier. This means that the binary relevance approach is simpler and less computationally expensive than the classifier chains approach. However, the classifier chains approach can capture label correlations better than the binary relevance approach.

## Obtaining a Multi-Label Classifier Using the Binary Relevance Approach

In [ ]:
# Read the data
data = pd.read_csv('yeast.csv')

data.drop(0)
X = data.iloc[:, :103].values 
y = data.iloc[:, 103:].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Create a base classifier
base_classifier = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Two hidden layers
    activation='relu',             # ReLU activation function
    solver='adam',                 # Adam optimizer
    max_iter=300,                  # Maximum iterations
    random_state=42,               # For reproducibility
    early_stopping=True,           # Enable early stopping
    validation_fraction=0.1        # Use 10% of training data for validation
)

# Create and train the binary relevance classifier
binary_relevance = BinaryRelevance(
    classifier=base_classifier,
    require_dense=[True, True]     # Both X and y should be dense matrices
)

# Train the model
binary_relevance.fit(X_train, y_train)

# Make predictions
y_pred = binary_relevance.predict(X_test)

# Convert sparse matrix predictions to dense array for evaluation
y_pred_dense = y_pred.toarray()
y_test_dense = y_test

print("Prediction type:", type(y_pred))
print("Prediction shape:", y_pred.shape if hasattr(y_pred, 'shape') else "no shape")
print("Dense prediction type:", type(y_pred_dense))
print("Dense prediction shape:", y_pred_dense.shape)
print("Test type:", type(y_test_dense))
print("Test shape:", y_test_dense.shape)

# Calculate performance metrics
accuracy = accuracy_score(y_test_dense, y_pred_dense)
hamming = hamming_loss(y_test_dense, y_pred_dense)
jaccard_score = jaccard_score(y_test_dense, y_pred_dense, average='samples')

print(f"Subset Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {hamming:.4f}")
print(f"Jaccard Score: {jaccard_score:.4f}")

In [ ]:
# Create a base classifier
base_classifier = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Two hidden layers
    activation='relu',             # ReLU activation function
    solver='adam',                 # Adam optimizer
    max_iter=300,                  # Maximum iterations
    random_state=42,              # For reproducibility
    early_stopping=True,          # Enable early stopping
    validation_fraction=0.1       # Use 10% of training data for validation
)

# Create and train the classifier chain
classifier_chain = ClassifierChain(
    classifier=base_classifier,
    require_dense=[True, True],    # Both X and y should be dense matrices
)

# Train the model
classifier_chain.fit(X_train, y_train)

# Make predictions
y_pred = classifier_chain.predict(X_test)

# Convert sparse matrix predictions to dense array for evaluation
y_pred_dense = y_pred.toarray()
y_test_dense = y_test

from sklearn.metrics import jaccard_score as js_metric

# Calculate performance metrics
accuracy = accuracy_score(y_test_dense, y_pred_dense)
hamming = hamming_loss(y_test_dense, y_pred_dense)
js = js_metric(y_test_dense, y_pred_dense, average='samples')

print(f"Subset Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {hamming:.4f}")
print(f"Jaccard Score: {js:.4f}")

# Adapted algorithm